# XGBoost 로컬 학습 후 Batch Transform

In [20]:
!pip install xgboost

     |████████████████████████████████| 173.5 MB 59.9 MB/s eta 0:00:01   |█████                           | 27.3 MB 1.6 MB/s eta 0:01:29
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [22]:
%%writefile source_dir/train.py

import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.keys())
print(boston.data.shape)
print(boston.feature_names)
print(boston.DESCR)

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data.head()
data['PRICE'] = boston.target

X, y = data.iloc[:,:-1], data.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                          colsample_bytree = 0.3,
                          learning_rate = 0.1,
                          max_depth = 5,
                          alpha = 10,
                          n_estimators = 10)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

Overwriting source_dir/train.py


In [43]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
boston = load_boston()
# print(boston.keys())
# print(boston.data.shape)
# print(boston.feature_names)
# print(boston.DESCR)

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data.head()
data['PRICE'] = boston.target

X, y = data.iloc[:,:-1], data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

train = xgb.DMatrix(X_train, y_train)
test = xgb.DMatrix(X_test, y_test)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                          colsample_bytree = 0.3,
                          learning_rate = 0.1,
                          max_depth = 5,
                          alpha = 10,
                          n_estimators = 10)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

params = {'objective':'reg:squarederror',
          'colsample_bytree': 0.3,
          'learning_rate': 0.1,
          'max_depth': 5,
          'alpha': 10}

# cv_results = xgb.cv(dtrain=data_dmatrix,
#                     params=params,
#                     nfold=3,
#                     num_boost_round=50,
#                     early_stopping_rounds=10,
#                     metrics="rmse",
#                     as_pandas=True, seed=123)

model = xgb.train(params=params,
                  dtrain=dtrain,
                  evals=[(train, "train"), (test, "validation")],
                  num_boost_round=50,
                  early_stopping_rounds=20)

with open('./model/model.pkl', 'wb') as f:
    pickle.dump(model, f)

RMSE: 10.915755
[0]	train-rmse:21.48545	validation-rmse:22.21274
[1]	train-rmse:19.59179	validation-rmse:20.35016
[2]	train-rmse:17.87300	validation-rmse:18.58683
[3]	train-rmse:16.31317	validation-rmse:16.99754
[4]	train-rmse:14.93596	validation-rmse:15.59446
[5]	train-rmse:13.72222	validation-rmse:14.32433
[6]	train-rmse:12.62987	validation-rmse:13.20950
[7]	train-rmse:11.55021	validation-rmse:12.05585
[8]	train-rmse:10.68027	validation-rmse:11.12384
[9]	train-rmse:9.79266	validation-rmse:10.18393
[10]	train-rmse:9.08310	validation-rmse:9.42841
[11]	train-rmse:8.47408	validation-rmse:8.83133
[12]	train-rmse:7.89037	validation-rmse:8.18891
[13]	train-rmse:7.27345	validation-rmse:7.62061
[14]	train-rmse:6.86154	validation-rmse:7.15773
[15]	train-rmse:6.49340	validation-rmse:6.73034
[16]	train-rmse:6.12574	validation-rmse:6.33825
[17]	train-rmse:5.72472	validation-rmse:5.89568
[18]	train-rmse:5.33770	validation-rmse:5.55162
[19]	train-rmse:5.00136	validation-rmse:5.19489
[20]	train-rmse

## Load model

In [44]:
loaded_model = pickle.load(open('./model/model.pkl', 'rb'))

In [47]:
preds = loaded_model.predict(y_train)

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.series.Series'>)